In [1]:
# 链接云端硬盘
import os
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/Shared drives/RunTu/KDD_CUP/code'
os.chdir(path)
os.listdir(path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


['preprocess.py',
 'model.py',
 'tracking.py',
 '.DS_Store',
 '__pycache__',
 '.ipynb_checkpoints',
 'tracking.py.gdoc',
 'Negative_Sampleling_data.csv',
 'Negative_Sampleling_data2.csv',
 'Negative_Sampleling_Whole_train_data.csv',
 'Data_Slicing.csv',
 'Data_Slicing_test.csv',
 'Negative_Sampleling.ipynb',
 'Recall_item_for_users.ipynb',
 'Deep_FM_whole_test.csv',
 'DeepFM.ipynb']

In [0]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
import sys
sys.path.append('../')

# 处理数据特征

In [0]:
!pip install deepctr[gpu]

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import deepctr
from deepctr.models import DeepFM
from deepctr.inputs import SparseFeat, DenseFeat, get_feature_names, VarLenSparseFeat

In [0]:
def reduce_mem(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(
            end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [0]:
train = pd.read_csv('Negative_Sampleling_data.csv')
test = pd.read_csv('/content/drive/Shared drives/RunTu/KDD_CUP/recommend_item_1')

In [7]:
train = reduce_mem(train)
test = reduce_mem(test)

Mem. usage decreased to 18.12 Mb (62.5% reduction)
Mem. usage decreased to 11.48 Mb (58.3% reduction)


#拼接item特征

In [0]:
# item 和user的特征名称
item_feat_cols = ["item_id"]+['text_vd' + str(i) for i in range(128)]+['img_vd' + str(i) for i in range(128)]

In [0]:
item_feat = pd.read_csv('/content/drive/Shared drives/RunTu/KDD_CUP/Data/underexpose_train/underexpose_item_feat.csv', names=item_feat_cols)

In [10]:
item_feat = reduce_mem(item_feat)

Mem. usage decreased to 56.09 Mb (73.7% reduction)


In [0]:
item_feat[item_feat_cols[1]] = item_feat[item_feat_cols[1]].apply(lambda x: x[1:])
item_feat[item_feat_cols[128]] = item_feat[item_feat_cols[128]].apply(lambda x: x[:-1])
item_feat[item_feat_cols[129]] = item_feat[item_feat_cols[129]].apply(lambda x: x[1:])
item_feat[item_feat_cols[256]] = item_feat[item_feat_cols[256]].apply(lambda x: x[:-1])

In [0]:
whole_train = train.merge(item_feat, on='item_id', how='left')
whole_test = test.merge(item_feat, on='item_id', how='left')

In [13]:
whole_trian = reduce_mem(whole_train)
whole_test = reduce_mem(whole_test)

Mem. usage decreased to 839.64 Mb (0.0% reduction)
Mem. usage decreased to 636.17 Mb (0.0% reduction)


In [0]:
whole_train.drop(['time'], axis=1, inplace=True)
whole_test.drop(['sim'], axis=1, inplace=True)

In [0]:
whole_train.dropna(axis=0, how='any', inplace=True)
whole_test.dropna(axis=0, how='any', inplace=True)

In [16]:
whole_train.head()

,user_id,item_id,click,text_vd0,text_vd1,text_vd2,text_vd3,text_vd4,text_vd5,text_vd6,text_vd7,text_vd8,text_vd9,text_vd10,text_vd11,text_vd12,text_vd13,text_vd14,text_vd15,text_vd16,text_vd17,text_vd18,text_vd19,text_vd20,text_vd21,text_vd22,text_vd23,text_vd24,text_vd25,text_vd26,text_vd27,text_vd28,text_vd29,text_vd30,text_vd31,text_vd32,text_vd33,text_vd34,text_vd35,text_vd36,...,img_vd88,img_vd89,img_vd90,img_vd91,img_vd92,img_vd93,img_vd94,img_vd95,img_vd96,img_vd97,img_vd98,img_vd99,img_vd100,img_vd101,img_vd102,img_vd103,img_vd104,img_vd105,img_vd106,img_vd107,img_vd108,img_vd109,img_vd110,img_vd111,img_vd112,img_vd113,img_vd114,img_vd115,img_vd116,img_vd117,img_vd118,img_vd119,img_vd120,img_vd121,img_vd122,img_vd123,img_vd124,img_vd125,img_vd126,img_vd127
0,1,38715,1.0,1.4344890117645264,0.337646,-0.946289,-3.054688,-0.367188,-0.984375,0.168091,0.081299,-2.154297,0.239868,-1.218750,-2.269531,1.828125,-0.593750,-1.792969,-2.228516,-0.082458,1.868164,-2.316406,-2.154297,-0.899414,-1.121094,1.947266,-0.576660,0.203979,2.070312,0.419922,0.551270,1.985352,-1.193359,-0.654297,-1.439453,-1.820312,-1.542969,0.735352,1.286133,-0.881348,...,1.777344,-1.172852,-0.119568,0.909180,2.207031,1.639648,-0.158447,0.504395,-0.684082,-2.117188,0.733398,1.372070,-1.658203,2.468750,1.383789,-0.727051,-1.208984,2.121094,1.713867,0.284424,-2.474609,0.456299,0.713379,0.284424,1.912109,-1.580078,1.344727,-0.111694,0.040894,1.029297,1.243164,-2.740234,0.202759,1.055664,2.867188,1.700195,3.052734,2.914062,2.714844,1.6945199966430664
1,1,54144,0.0,-1.6079577207565308,-1.041016,0.290039,-1.550781,0.206665,2.710938,2.296875,0.452148,-3.111328,-2.341797,1.277344,-2.880859,-1.071289,0.364502,0.960938,0.913086,-2.925781,-3.832031,-0.548828,-2.400391,-1.354492,1.079102,2.234375,-0.938477,-1.123047,-0.135254,2.134766,0.736816,-2.878906,2.609375,0.406250,0.222046,-0.979004,2.875000,1.017578,4.398438,4.910156,...,2.216797,-1.420898,-2.990234,1.020508,0.075928,1.051758,-0.595703,-0.688477,2.666016,0.689941,-1.831055,-2.671875,-0.452148,1.649414,0.659180,1.981445,-0.169800,-1.330078,0.018555,0.591797,-3.128906,-2.492188,0.798340,-0.102417,-0.193115,1.535156,0.142700,-0.860840,5.171875,-0.286377,1.143555,-2.046875,0.885254,-2.121094,0.248535,1.820312,-0.216797,2.119141,3.007812,-0.004481494426727295
2,1,106971,0.0,4.284095287322998,-3.558594,-1.075195,-2.351562,-4.082031,2.248047,2.931641,1.208008,-4.433594,1.850586,-1.231445,0.257324,0.546875,-0.217896,4.820312,-3.117188,0.246826,-1.391602,-4.566406,0.270264,-3.197266,-0.320557,0.276123,-2.863281,-0.244141,1.885742,0.660156,1.496094,1.948242,3.603516,-0.752930,-1.102539,1.566406,-3.109375,2.328125,0.414795,1.217773,...,0.939941,-1.658203,-1.118164,-0.244263,2.605469,-3.738281,0.593262,0.152100,4.261719,0.858398,1.128906,-0.197632,-0.597168,3.935547,-1.787109,2.617188,2.257812,2.953125,0.768066,-1.493164,-2.277344,0.637207,-1.328125,1.408203,2.599609,0.275879,-2.355469,0.541504,1.151367,0.728027,-0.723145,-2.755859,2.064453,0.867188,2.003906,1.379883,2.908203,3.400391,3.265625,-0.2695731222629547
3,1,30964,0.0,3.4569389820098877,-0.514160,-1.311523,-3.460938,0.972656,2.957031,1.890625,-0.864746,-4.683594,1.833984,1.360352,-3.578125,2.015625,-4.132812,-5.601562,-3.853516,-0.066345,-1.564453,-2.978516,-0.030746,-2.128906,-2.570312,4.042969,3.441406,-3.419922,-6.175781,0.635254,-4.621094,-2.566406,1.697266,-4.617188,-1.284180,-4.058594,2.406250,1.750000,-2.820312,-4.445312,...,2.621094,-2.748047,-2.291016,-0.727051,-0.906738,-1.940430,3.574219,-0.762695,-0.811035,-0.652344,0.905273,-0.767578,-1.226562,3.919922,0.273193,-1.687500,-0.078979,-0.676270,0.984863,0.328125,-3.232422,-0.354980,0.283691,0.509277,-1.008789,-3.033203,1.470703,-0.675293,-0.112244,0.185547,-0.191650,-3.435547,1.497070,-0.832031,-2.244141,0.061340,1.393555,3.330078,3.671875,1.2184604406356812
4,1,20867,0.0,0.7406136989593506,-1.109375,0.057800,-3.787109,-4.207031,3.476562,-0.734375,0.199707,-2.585938,1.070312,-1.076172,-4.457031,2.822266,0.8

In [17]:
whole_test.head()

,user_id,item_id,text_vd0,text_vd1,text_vd2,text_vd3,text_vd4,text_vd5,text_vd6,text_vd7,text_vd8,text_vd9,text_vd10,text_vd11,text_vd12,text_vd13,text_vd14,text_vd15,text_vd16,text_vd17,text_vd18,text_vd19,text_vd20,text_vd21,text_vd22,text_vd23,text_vd24,text_vd25,text_vd26,text_vd27,text_vd28,text_vd29,text_vd30,text_vd31,text_vd32,text_vd33,text_vd34,text_vd35,text_vd36,text_vd37,...,img_vd88,img_vd89,img_vd90,img_vd91,img_vd92,img_vd93,img_vd94,img_vd95,img_vd96,img_vd97,img_vd98,img_vd99,img_vd100,img_vd101,img_vd102,img_vd103,img_vd104,img_vd105,img_vd106,img_vd107,img_vd108,img_vd109,img_vd110,img_vd111,img_vd112,img_vd113,img_vd114,img_vd115,img_vd116,img_vd117,img_vd118,img_vd119,img_vd120,img_vd121,img_vd122,img_vd123,img_vd124,img_vd125,img_vd126,img_vd127
0,1133,99136,1.8157325983047485,2.062500,0.316895,-1.930664,-1.330078,-3.080078,-0.978027,0.462402,-5.554688,-2.734375,2.355469,-0.600586,2.527344,3.162109,-1.012695,0.170044,0.871094,0.258057,0.811035,1.913086,-2.550781,-1.686523,0.445801,0.648926,2.847656,0.565918,3.335938,-1.797852,-1.405273,0.618652,-1.590820,-1.888672,-2.314453,0.615723,1.275391,2.382812,-1.535156,-5.546875,...,0.411865,-1.498047,0.090515,-2.406250,-1.295898,-1.243164,3.988281,2.087891,2.011719,3.453125,-2.193359,0.563477,1.072266,3.033203,-0.075073,0.163208,-4.226562,-1.892578,3.210938,-0.539551,-2.548828,-0.942871,-1.369141,2.451172,-1.068359,-2.736328,0.885742,-4.386719,0.496826,-0.835938,-1.010742,-1.114258,-0.120911,-1.139648,-2.160156,-2.773438,0.430176,3.390625,1.487305,1.3710342645645142
1,1133,87420,1.9625258445739746,2.607422,1.395508,-4.011719,-1.791992,-2.462891,-1.127930,-1.356445,-4.675781,-3.654297,2.996094,-1.571289,3.650391,2.505859,-2.265625,0.520020,2.185547,0.390137,2.998047,1.182617,-2.023438,-1.971680,1.541016,1.620117,1.042969,0.650391,4.859375,-2.117188,-2.230469,1.801758,-2.669922,-0.925781,-2.980469,0.031403,1.447266,2.968750,-2.939453,-4.093750,...,4.320312,-0.426514,0.577148,-2.625000,0.555176,0.195557,3.197266,2.292969,2.216797,3.654297,-2.894531,-0.257080,-0.171875,2.312500,1.576172,-2.052734,-3.601562,-3.992188,0.856445,1.171875,-2.199219,-2.943359,-2.066406,1.879883,-2.927734,-0.678223,-0.356201,-2.332031,-0.511230,1.005859,-1.744141,-2.587891,2.693359,-1.600586,-2.539062,-2.833984,1.812500,4.015625,3.222656,-0.6276818513870239
2,1133,72322,-0.22904649376869202,-1.307617,0.381836,-0.514648,2.083984,-0.736328,-2.998047,1.797852,-4.175781,0.670410,-0.125000,-0.296875,3.246094,0.343994,-2.033203,0.129395,0.581543,-0.687500,-1.121094,-1.546875,0.566406,-2.775391,0.362549,0.188599,-0.425537,-1.599609,3.541016,-3.363281,-0.307373,1.827148,-2.552734,-2.601562,0.685059,0.008644,3.292969,-1.206055,-3.712891,-2.613281,...,1.420898,-1.789062,0.686035,0.762207,4.585938,-1.101562,2.595703,-1.258789,3.630859,2.027344,1.775391,0.668457,2.916016,1.283203,0.942383,-0.750977,0.199463,2.310547,1.200195,-2.578125,-1.187500,-1.787109,-1.192383,0.245972,-0.330078,1.092773,0.154175,-0.787109,-2.996094,-1.808594,-1.487305,-1.981445,3.900391,0.635742,-0.729004,0.079224,3.468750,0.697266,3.275391,-3.098668336868286
3,1133,88603,0.7165994644165039,4.050781,-2.435547,-2.609375,-4.425781,-2.605469,-1.506836,0.870605,-5.023438,-3.074219,0.363525,-1.698242,3.367188,4.105469,-0.438477,1.642578,-0.719727,-0.210693,2.884766,1.261719,-3.773438,-0.817871,2.087891,0.394775,1.400391,2.617188,4.257812,0.573242,0.999023,1.653320,-0.947266,-3.218750,-3.173828,0.379639,2.841797,1.828125,-2.265625,-4.550781,...,0.152588,-1.767578,3.562500,-1.809570,0.630371,4.214844,1.435547,1.161133,0.507324,2.988281,-2.007812,1.507812,2.587891,0.463623,3.216797,4.828125,-2.275391,0.238770,1.056641,-2.902344,-1.264648,-2.560547,3.919922,1.019531,0.166382,-5.562500,1.341797,-2.533203,0.192505,-1.276367,1.144531,-0.317383,0.006886,-1.813477,1.898438,3.546875,1.679688,0.957031,1.419922,0.37967124581336975
4,1133,113941,0.8262506127357483,-2.039062,0.104919,-0.642090,-0.049164,1.741211,-0.299072,-0.197998,-1.690430,-0.271

# 数据预处理传入模型

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
cate_cols = ['user_id','item_id']
drop_cols = ['click']

In [0]:
sparse_features = cate_cols
dense_features = [i for i in whole_train.columns if ((i not in cate_cols) & (i not in drop_cols))]

In [0]:
for feat in sparse_features:
    lbe = LabelEncoder()
    whole_train[feat] = lbe.fit_transform(whole_train[feat])

In [0]:
# 1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
    lbe = LabelEncoder()
    whole_train[feat] = lbe.fit_transform(whole_train[feat])

mms = MinMaxScaler(feature_range=(0, 1))
whole_train[dense_features] = mms.fit_transform(whole_train[dense_features])

In [0]:
# 2.count #unique features for each sparse field,and record dense feature field name
# SparseFeat('name', 'vocabulary_size', 'embedding_dim', 'use_hash', 'dtype', 'embedding_name','group_name')
# DenseFeat('name', 'dimension', 'dtype')
fixlen_feature_columns = [SparseFeat(feat, whole_train[feat].nunique())for feat in sparse_features] + [DenseFeat(feat, 1, )for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
print(feature_names)

In [0]:
target = whole_train['click']
# 3.generate input data for model
train, test, train_label, test_label = train_test_split(whole_train, target, test_size=0.2)
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}

In [0]:
# 4.Define Model,train,predict and evaluate
model = DeepFM(linear_feature_columns, dnn_feature_columns, l2_reg_dnn=0.01, dnn_dropout=0.5, task='binary')

model.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'], )

history = model.fit(train_model_input, train_label, batch_size=128, epochs=1, verbose=2, validation_split=0.2)

pred_ans = model.predict(test_model_input, batch_size=128)

# print("test LogLoss", round(log_loss(test_label, pred_ans), 4))
print("test AUC", round(roc_auc_score(test_label, pred_ans), 4))

# Predict

In [0]:
# 处理测试集
for feat in sparse_features:
    lbe = LabelEncoder()
    whole_test[feat] = lbe.fit_transform(whole_test[feat])

mms = MinMaxScaler(feature_range=(0, 1))
whole_test[dense_features] = mms.fit_transform(whole_test[dense_features])

In [0]:
fixlen_feature_columns = [SparseFeat(feat, whole_test[feat].nunique())for feat in sparse_features] + [DenseFeat(feat, 1, )for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
print(feature_names)

In [0]:
predict_input = {name: whole_test[name] for name in feature_names}

In [0]:
result = model.predict(predict_input, batch_size=64)